In [29]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from testCases_v2 import *
from dnn_utils_v2 import sigmoid, sigmoid_backward, relu, relu_backward

***2 Layer Neural Network***

In [37]:
def initialize_parameters(n_x, n_h, n_y):

    np.random.seed(1)

    W1= np.random.randn(n_h, n_x) * 0.01
    b1= np.zeros(shape=(n_h, 1))
    W2= np.random.randn(n_y, n_h) * 0.01
    b2= np.zeros(shape=(n_y,1))

    assert(W1.shape == (n_h, n_x))
    assert(b1.shape == (n_h, 1))
    assert(W2.shape == (n_y, n_h))
    assert(b2.shape == (n_y, 1))

    parameters = {
        "W1": W1, 
        "b1" : b1, 
        "W2" : W2, 
        "b2" : b2, 
        
    }
    
    return parameters

In [51]:
parameters = initialize_parameters(2,2, 1)
print("W1 is:" + str(parameters['W1']))
print("b1 is:" + str(parameters['b2']))
print("W2 is:" + str(parameters['W2']))
print("b2 is:" + str(parameters['b2']))

W1 is:[[ 0.01624345 -0.00611756]
 [-0.00528172 -0.01072969]]
b1 is:[[0.]]
W2 is:[[ 0.00865408 -0.02301539]]
b2 is:[[0.]]
